# Análise de Dados Soccer2D

## Introdução

Esta oficina proporcionará uma introdução prática e direta à análise descritiva de dados no contexto de futebol de robôs, utilizando dados coletados de partidas da RoboCup Soccer Simulation 2D, que é uma competição em que dois times de 11 robôs autônomos simulados joguem uma partida de futebol contra outros times.

A pasta `data` contém os arquivos csv das partidas do round final da edição de 2023 da RoboCup.

## Objetivos

1. Processar os dados de uma partida;
2. Visualizar os dados processados;
3. Identificar algumas das estratégias e o estilo de jogo de ambas as equipes;

Antes de iniciarmos as análises, nós precisamos que algumas bibliotecas já estejam disponíveis. São elas:

- [Numpy](http://www.numpy.org/) para todas as operações matriciais;
- [Matplotlib](https://matplotlib.org/) para a plotagem dos gráficos;
- [Pandas](https://pandas.pydata.org/docs/index.html) que contém funções e ferramentas de manipulação e visualização de dados.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Exploração dos dados

Nesta etapa, carregaremos o dataset e, em seguida, detalharemos as informações que ele possui.

In [3]:
# ajuste o path para a localização do arquivo csv
df = pd.read_csv('data/Oxsy_1-vs-YuShan2023_3.csv')

In [ ]:
# visualizando os dados
df.head()

In [ ]:
# visualizando todas as colunas e uma lista
df.columns.to_list()

As colunas do dataset contém informações acerca da partida e de cada um dos jogadores de cada time. Algumas das colunas que utilizaremos nesta análise podem ser descritas da seguinte forma:

partida:
- show_time: ciclo do servidor (~ 0.1 segundo)
- playmode: estado atual do jogo (escanteio, pênalti, falta, etc)
- team_name_[LADO]: nome do time que está em cada lado
- team_score_[LADO]: gols de cada time
- team_pen_score_[LADO]: gols de pênalti
- team_pen_miss_[LADO]: gols de pênalti perdidos
- ball_[COORD]: coordenada x ou y da bola
- ball_v[COORD]: velocidade da bola no eixo x ou y

jogadores:
- player_[LADO][NUM_JOG]_unum: número da camisa do jogador. 0 significa que o jogador está desativado 
- player_[LADO][NUM_JOG]_type: id do tipo de cada jogador (meio-campista, defensor, goleiro, etc.)
- player_[LADO][NUM_JOG][COORD]: coordenada x ou y do jogador
- player_[LADO][NUM_JOG]_v[COORD]: velocidade do jogador
- player_[LADO][NUM_JOG]_body: angulação do corpo do jogador, variando no intervalo de (-180, 180) graus
- player_[LADO][NUM_JOG]_neck: angulação do pescoço do jogador, variando no intervalo de (-90, 90) graus
- player_[LADO][NUM_JOG]_attribute_stamina: nível atual de stamina do jogador
- player_[LADO][NUM_JOG]_attribute_stamina_capacity: capacidade total de stamina do jogador
- player_[LADO][NUM_JOG]_counting_kick: quantidade de chutes do jogador
- player_[LADO][NUM_JOG]_counting_dash: quantidade de carrinhos realizados pelo jogador
- player_[LADO][NUM_JOG]_counting_catch: quantidade de bolas agarradas pelo goleiro (para outros jogadores será sempre 0)
- player_[LADO][NUM_JOG]_counting_tackle: quantidade de bolas tomadas pelo do jogador

Além dos dados descritos acima, há informações sobre atributos dos jogadores e contagem do número de ações de cada jogador, como faltas, chutes, defesas, etc.

A coluna `playmode` descreve modos do jogo que podem ocorrer durante a partida. Cada modo corresponde à um evento, semelhante ao futebol da vida real.

In [ ]:
df['playmode'].unique()

Segue a descrição de cada modo de jogo:

- **kick_off_[LADO]**: Início da partida com o time à esquerda ou direita dando o pontapé inicial.
- **play_on**: O jogo está em andamento normalmente, sem interrupções.
- **kick_in_[LADO]**: O time à direita ou esquerda executa um arremesso lateral.
- **goal_[LADO]**: Gol marcado pelo time à direita ou esquerda.
- **foul_charge_[LADO]**: Falta cometida pelo time à esquerda ou direita por carga irregular.
- **free_kick_[LADO]**: Cobrança de falta para o time à direita ou esquerda.
- **kick_off_[LADO]**: Início da partida ou reinício com o time à direita ou esquerda dando o pontapé inicial.
- **offside_[LADO]**: Impedimento do time à direita ou esquerda.
- **goal_kick_[LADO]**: Tiro de meta cobrado pelo time à direita ou esquerda.
- **corner_kick_[LADO]**: Cobrança de escanteio pelo time à esquerda ou direita.
- **time_over**: O tempo regulamentar do jogo terminou.

## 2. Processando os dados

### Funções e classes auxiliares

Antes de começarmos a análise, vale definir algumas funções e classes que podem ser úteis durante o processo.

Para simplificar, optamos por criar um arquivo chamado `geom_2d` que contém implementação de abstrações geométricas, como o ponto e o círculo, além de funções como o cálculo de distância entre dois pontos.

Também criamos funções auxiliares no arquivo `helpers.py` que irão extrair e processar dados úteis a partir dos dados coletados, como estimar a posse de bola a partir da posição dos jogadores.

No entanto, recomendamos fortemente que você abra e entenda o código desses arquivos para ampliar o seu entendimento da análise como um todo. Todos os códigos disponibilizados foram implementados e comentados para que o seu entendimento seja facilitado.

In [7]:
from geom_2d import Point, Circle, distance
from helpers import *

Para reunir todos os dados processados que obteremos nesta etapa, criaremos um novo dataset.

In [8]:
game_stats = pd.DataFrame()

#### Posse de bola

Para calcular a posse de bola, basta calcular a porcentagem do tempo total em que um determinado time esteve com a bola.

$$P_{t} = \frac{T_{t}}{T_{Total}} * 100$$

Consideraremos que o time mais próximo da bola terá o domínio.

In [9]:
# colunas referentes às coordenadas de cada jogador
players_left = []
players_right = []
for i in range(1, 12):
    players_left.append((f'player_l{i}_x', f'player_l{i}_y'))
    players_right.append((f'player_r{i}_x', f'player_r{i}_y'))
players = [players_left, players_right]

Para o cálculo do tempo em que cada time esteve com a posse da bola, iremos iterar por cada ciclo do servidor, contando os ciclos em que cada time esteve sob o domínio da bola. 

Utilizaremos a função `possession_at` para determinar de quem é a posse a cada ciclo. A implementação desta função se encontra no arquivo `helpers.py`. 

In [ ]:
# momentos em que o jogo não estava parado
filtered_game = df[df['playmode'] == 'play_on']

# inicializa os contadores de tempo
time_left = time_right = 0

# inicializa uma lista para armazenar a posse a cada instante
possession_data = []

for cycle, row in filtered_game.iterrows():
  # função auxiliar possession_at() para determinar a posse
  possession_side = possession_at(cycle, filtered_game, players)

  if possession_side == 'left':
    time_left += 1
  else:
    time_right += 1

  # cálculo das posses
  total_time = time_left + time_right
  possession_left = time_left/ total_time
  possession_right = time_right/ total_time

  # armazena a posse a cada instante
  possession_data.append({'cycle': cycle, 'possession_l': possession_left, 'possession_r': possession_right})

# converte a lista para um DataFrame e adiciona ao game_stats
possession_df = pd.DataFrame(possession_data)
game_stats = pd.concat([game_stats, possession_df], axis=1)

team_name_left = df['team_name_l'][0]
team_name_right = df['team_name_r'][0]

# posse no formato tabelado
possession_summary = pd.DataFrame({
  'Time': [team_name_left, team_name_right],
  'Posse (%)': [possession_left * 100, possession_right * 100]
})

print("Posse de bola")
print("==========================")
print(possession_summary.to_string(index=False))
print("==========================")

Podemos gerar visualizações da posse de bola para extrair insights úteis

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(game_stats['cycle'], game_stats['possession_l'], label=team_name_left)
plt.plot(game_stats['cycle'], game_stats['possession_r'], label=team_name_right)
plt.xlabel('Ciclo')
plt.ylabel('Posse')
plt.title('Posse de bola x Ciclos')
plt.legend(loc='upper right')
plt.show()

#### Passes

Para contabilizar os passes de cada time, precisaremos iterar por cada ciclo do servidor, monitorando os passes e mudanças de posse de bola, enquanto rastreamos o número de passes corretos, errados e interceptados para ambos os times.

**Para cada ciclo do jogo**:
- Verifica-se se um jogador do time direito ou esquerdo fez um passe.
- Se um passe está em andamento, determinamos a posse da bola e atualizamos os contadores de passes corretos, errados ou interceptados com base na continuidade ou interrupção da posse.
- Para passes interceptados, verifica-se se houve interrupções no jogo (`kick_off` ou `foul_charge`) nos ciclos seguintes, ajustando o contador se necessário.

O objetivo final é contabilizar a eficácia dos passes e a frequência de interceptações durante o jogo.

In [12]:
# time esquerdo
correct_passes_l, wrong_passes_l, intercepted_passes_l = analyse_passes(df, players, team_name_left)

# time direito
correct_passes_r, wrong_passes_r, intercepted_passes_r = analyse_passes(df, players, team_name_right)

Ao final do loop, basta calcular a precisão dos passes e mostar os resultados.

In [ ]:

# Calcula o total de passes e a precisão de passes para ambos os times
left_team_total_passes = correct_passes_l + wrong_passes_l + intercepted_passes_r
right_team_total_passes = correct_passes_r + wrong_passes_r + intercepted_passes_l

left_team_passing_accuracy = correct_passes_l / left_team_total_passes
right_team_passing_accuracy = correct_passes_r / right_team_total_passes

# Atribui os valores finais para passes completos e interceptações
left_team_completed_passes = correct_passes_l
right_team_completed_passes = correct_passes_r

left_team_interceptions = intercepted_passes_l
right_team_interceptions = intercepted_passes_r

print(
f"""
==========================
{team_name_left}

passes totais: {left_team_total_passes}
passes completos: {left_team_completed_passes}
acurácia: {left_team_passing_accuracy:.2f}
interceptações: {left_team_interceptions}

==========================
{team_name_right}

passes totais: {right_team_total_passes}
passes completos: {right_team_completed_passes}
acurácia: {right_team_passing_accuracy:.2f}
interceptações: {right_team_interceptions}

===========================
"""
)

Com base nos dados obtidos nas células anteriores, podemos comparar a distribuição de passes de cada time.

In [ ]:
# quantidade de passes incompletos
left_team_incompleted_passes = left_team_total_passes - left_team_completed_passes
right_team_incompleted_passes = right_team_total_passes - right_team_completed_passes

# parâmetros do gráfico 
labels = ['Passes Completos', 'Passes Errados', 'Passes Interceptados']
explode = (0, 0.1, 0.05) # destaca a fatia de passes incompletos
colors = ['#4CAF50', '#FF6347', '#1E90FF']

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# gráfico de pizza para o time esquerdo
sizes = [left_team_completed_passes, left_team_incompleted_passes, intercepted_passes_l]
axs[0].pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
axs[0].set_title(f'Passes Completos vs Incompletos - {team_name_left}')

# gráfico de pizza para o time direito
sizes_right = [right_team_completed_passes, right_team_incompleted_passes, intercepted_passes_r]
axs[1].pie(sizes_right, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
axs[1].set_title(f'Passes Completos vs Incompletos - {team_name_right}')

plt.show()

In [ ]:
# dados a serem mostrados no histograma
passes_data = {
    'Passes Corretos': [correct_passes_l, correct_passes_r],
    'Passes Errados': [wrong_passes_l, wrong_passes_r],
    'Passes Interceptados': [intercepted_passes_l, intercepted_passes_r]
}

# nome dos times
teams = [team_name_left, team_name_right]

# plotando o histograma
fig, ax = plt.subplots(figsize=(10, 6))

# tamanho da barra
bar_width = 0.25

# posição das barras no eixo x
r1 = np.arange(len(passes_data['Passes Corretos']))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]

# passes totais
total_passes = [left_team_total_passes, right_team_total_passes]
r4 = [x + bar_width for x in r3]

# criando as barras
ax.bar(r1, total_passes, color='#fadb44', width=bar_width, edgecolor='grey', label='Passes totais')
ax.bar(r2, passes_data['Passes Corretos'], color='g', width=bar_width, edgecolor='grey', label='Passes Corretos')
ax.bar(r3, passes_data['Passes Errados'], color='r', width=bar_width, edgecolor='grey', label='Passes Errados')
ax.bar(r4, passes_data['Passes Interceptados'], color='b', width=bar_width, edgecolor='grey', label='Passes Interceptados')

# labels
ax.set_xlabel('Times', fontweight='bold')
ax.set_ylabel('Número de Passes', fontweight='bold')
ax.set_title('Distribuição dos passes', fontweight='bold')
ax.set_xticks([r + bar_width for r in range(len(passes_data['Passes Corretos']))])
ax.set_xticklabels(teams)

ax.legend()

plt.show()

### Escanteios

Como comentado anteriormente, a coluna `playmode` especifica os estados do jogo a cada ciclo. Podemos contar a ocorrência única de cada um desses estados durante o jogo por meio da função `find_last_unique_event_ocurrences`.

In [ ]:
# retorna uma lista com o ciclo em que o escanteio ocorreu para cada time
left_occurrencies = find_last_unique_event_ocurrences(df, "corner_kick_l")
right_occurrencies = find_last_unique_event_ocurrences(df, "corner_kick_r")

print(f"momentos dos escanteios do {team_name_left}: {left_occurrencies}")
print("==================================================")
print(f"momentos dos escanteios do {team_name_right}: {right_occurrencies}")

### Faltas

Para analisar as faltas, iremos iterar por cada ciclo, verificando se uma falta ocorreu. Caso positivo, iremos armazenar o local onde cada falta ocorreu. A função `analyze_fouls` encapsula essa lógica. 

In [ ]:
left_charges, right_charges = analyze_fouls_charge(df)

print(f"posições das faltas do {team_name_left}:")
for f in left_charges:
    print(f)

print(f"posições das faltas do {team_name_right}:")
for f in right_charges:
    print(f)

Podemos visualizar graficamente as posições das faltas no campo utilizado pela competição.

![Field Landmarks](images/landmarks.png)

Note que podemos utilizar as marcações do campo acima, por meio da classe `landmarks`, que lista algumas delas. 

In [18]:
from landmarks import Landmarks

In [ ]:
# Carrega a imagem do campo
field_img = plt.imread('images/sim2d_field.png')

fig, ax = plt.subplots(figsize=(10, 6))

# Limites do campo
left = -57.5
right = 57.5
top = 34.
bottom = -34.
limits = [left, right, top, bottom]

# Imagem do campo
ax.imshow(field_img, extent=limits)

# Posições das faltas
left_charge_positions = [(charge.x, charge.y) for charge in left_charges]
right_charge_positions = [(charge.x, charge.y) for charge in right_charges]

# Plot
if left_charge_positions:
    left_x, left_y = zip(*left_charge_positions)
    ax.scatter(left_x, left_y, color='blue', label=f'Faltas cometidas {team_name_left}')

if right_charge_positions:
    right_x, right_y = zip(*right_charge_positions)
    ax.scatter(right_x, right_y, color='red', label=f'Faltas cometidas {team_name_right}')

# Set labels and title
ax.set_xlabel('Coordenada X')
ax.set_ylabel('Coordenada Y')
ax.set_title('Posições das Faltas')
ax.legend()

# Show the plot
plt.show()

### Defesas do Goleiro

Podemos analisar o desempenho dos goleiros de cada time por meio da função `analyse_goalkeeper`, que irá extrair a quantidade de defesas, gols concedidos, a distância do goleiro para a bola e a posição de ambos, respectivamente.

In [ ]:
catches_l, goals_r, distances_l, ball_positions_l, goalie_positions_l = analyze_goalkeeper(df, players, team_name_left)
catches_r, goals_l, distances_r, ball_positions_r, goalie_positions_r = analyze_goalkeeper(df, players, team_name_right)

print(
    f"""
    Team {team_name_left} goalie's stats:

    catches: {catches_l}
    goals given: {goals_r}
    distances of the goalkeeper from the ball: {distances_l}
    positions of the goalie when the goals occured: {goalie_positions_l}
    positions of the ball when the goals occured: {ball_positions_l}

    ====================================================================
    
    Team {team_name_right} goalie's stats:

    catches: {catches_r}
    goals given: {goals_l}
    distances of the goalkeeper from the ball: {distances_r}
    positions of the goalie when the goals occured: {goalie_positions_r}
    positions of the ball when the goals occured: {ball_positions_r}
    """    
)

In [ ]:
# Quantidade de gols concedidos vs defesas
left_team_goals_given = goals_r
left_team_catches = catches_l

right_team_goals_given = goals_l
right_team_catches = catches_r

# Dados a serem plotados
teams = [team_name_left, team_name_right]
goals_given = [left_team_goals_given, right_team_goals_given]
catches = [left_team_catches, right_team_catches]

# Histograma
fig, ax = plt.subplots(figsize=(10, 6))

bar_width = 0.35
r1 = np.arange(len(teams))
r2 = [x + bar_width for x in r1]

ax.bar(r1, goals_given, color='r', width=bar_width, edgecolor='grey', label='Gols concedidos')
ax.bar(r2, catches, color='g', width=bar_width, edgecolor='grey', label='Defesas')

ax.set_xlabel('Times', fontweight='bold')
ax.set_ylabel('Quantidade', fontweight='bold')
ax.set_title('Quantidade de Gols concedidos vs Defendidos', fontweight='bold')
ax.set_xticks([r + bar_width / 2 for r in range(len(teams))])
ax.set_xticklabels(teams)

ax.legend()

plt.show()


### Chutes a gol

Para a extração da quantidade de chutes a gols, utilizaremos uma estratégia semelhante à extração de dados anteriores, como faltas, escanteios e passes.

Desse modo, utilizaremos a classe `Shooting`, que reúne todos os métodos e estatísticas relacionados à chutes a gol.

In [22]:
from shooting import Shoot

In [ ]:
shoot_analyser = Shoot(df)

# time esquerdo
shots_l = shoot_analyser.get_total_team_shots('l')
shots_on_target_l = shoot_analyser.get_team_on_target_shots('l')

# time direito
shots_r = shoot_analyser.get_total_team_shots('r')
shots_on_target_r = shoot_analyser.get_team_on_target_shots('r')

print(
    f""" 
    Team {df['team_name_l'][0]} shot stats:

    total shots: {shots_l}
    shots on target: {shots_on_target_l}
    accuracy: {shots_on_target_l/shots_l * 100:.1f}
    
    ==========================================
    
    Team {df['team_name_r'][0]} shot stats:

    total shots: {shots_r}
    shots on target: {shots_on_target_r}
    accuracy: {shots_on_target_r/shots_r * 100:.1f}
    """
)

In [ ]:
teams = [team_name_left, team_name_right]
total_shots = [shots_l, shots_r]
shots_on_target = [shots_on_target_l, shots_on_target_r]

fig, ax = plt.subplots(figsize=(10, 6))

# bar width
bar_width = 0.35

# posições das barras no eixo x
r1 = np.arange(len(total_shots))
r2 = [x + bar_width for x in r1]

ax.bar(r1, total_shots, color='b', width=bar_width, edgecolor='grey', label='Total Shots')
ax.bar(r2, shots_on_target, color='g', width=bar_width, edgecolor='grey', label='Shots on Target')

ax.set_xlabel('Teams', fontweight='bold')
ax.set_ylabel('Number of Shots', fontweight='bold')
ax.set_title('Total Shots vs Shots on Target', fontweight='bold')
ax.set_xticks([r + bar_width / 2 for r in range(len(total_shots))])
ax.set_xticklabels(teams)

ax.legend()

for i in range(len(total_shots)):
    ax.text(r1[i], total_shots[i] + 0.5, str(total_shots[i]), ha='center', fontweight='bold')
    ax.text(r2[i], shots_on_target[i] + 0.5, str(shots_on_target[i]), ha='center', fontweight='bold')

plt.show()

### Gols esperados(xG)

Para cada chute a gol, é possível calcular a probabilidade do chute resultar em um gol, dado um momento específico. A soma da probabilidade de gols esperados para cada chute é chamada de **xG score**, que reflete a dinâmica de uma partida e torna possível identificar qual time teve as melhores chances. 

O cálculo da probabilidade de gols esperados é dado pela aplicação de um modelo matemático que se ajusta à frequência de gols observada.






In [ ]:
shoot_analyser.describe()

### Gasto de Stamina

A stamina representa o vigor físico de cada jogador. Podemos extrair essa informação do dataset por meio da coluna `player_[LADO][NUM]_attribute_stamina`.

In [ ]:
team_l_stamina, team_r_stamina = analyze_stamina(df)

# gasto médio de stamina por jogador
stamina_avg_l = calculate_stamina_avg(team_l_stamina) 
stamina_avg_r = calculate_stamina_avg(team_r_stamina)

# gasto médio de stamina por time
total_stamina_avg_l = sum(stamina_avg_l)/11
total_stamina_avg_r = sum(stamina_avg_r)/11

print(f"""Custo médio de stamina por time:
      
{team_name_left}: {total_stamina_avg_l:.2f}
================================
{team_name_right}: {total_stamina_avg_r:.2f}""")


In [ ]:
teams = [team_name_left, team_name_right]
total_stamina_avg = [total_stamina_avg_l, total_stamina_avg_r]

bar_width = 0.55

plt.figure(figsize=(8, 6))
plt.bar(teams, total_stamina_avg, width=bar_width, color=['blue', 'red'])
plt.xlabel('Times')
plt.ylabel('Gasto Médio de Stamina')
plt.title('Gasto Médio de Stamina entre os times')
plt.show()

Uma forma melhor de visualizar o gasto médio de stamina é por meio de um box plot ou um gráfico de caixa.

No gráfico, a haste vertical é interpretada de baixo para cima, sendo que a parte inferior indica o mínimo e a superior indica o máximo, sempre desconsiderando possíveis outliers.

O retângulo central da haste possui três linhas que estão na horizontal: a linha de baixo, representada pelo contorno externo inferior do retângulo, indica o primeiro quartil. A linha de cima, que é o contorno externo superior do retângulo, indica o terceiro quartil. Já a linha interna indica a mediana ou o chamado segundo quartil.

In [ ]:
data = [stamina_avg_l, stamina_avg_r]

labels = [team_name_left, team_name_right]

plt.figure(figsize=(10, 6))
plt.boxplot(data, tick_labels=labels)
plt.xlabel('Times')
plt.ylabel('Custo médio de Stamina')
plt.title('Custo Médio de Stamina de cada time')
plt.show()

## 3 Desafios

Nesta seção, você deve realizar responder as seguintes perguntas analisando qualquer uma das partidas disponíveis em `data`. Utilize os gráficos e funções apresentadas anteriormente como base para suas próprias análises.

##### 3.1 Analisando a posse de bola, é possível determinar quais eventos do jogo causaram maiores variações na posse de bola?  

##### 3.2 Qual time foi mais ofensivo e qual foi mais defensivo? Utilize gráficos e estatísticas que comprovem seus argumentos.

##### 3.3 Analise os outros dois jogos do time *YuShan2023*. Existem diferenças significativas na distribuição da posse de bola, de passes corretos e do xG?

##### 3.4 Analise a posição da bola nos 100 ciclos anteriores a um gol e esboce a trajetória da bola no campo.

##### 3.5 Verifique a relação entre a quantidade de passes corretos e a taxa de gols esperados.

### Referências

- [(Artigo) A Data Analysis Graphical user Interface for RoboCup 2D Soccer Simulation League](https://ieeexplore.ieee.org/document/9307104)
- [SoccerAnalyser](https://github.com/robocin/SoccerAnalyzer)
- [Python Datascience Handbook](https://github.com/jakevdp/PythonDataScienceHandbook/blob/8a34a4f653bdbdc01415a94dc20d4e9b97438965/notebooks/Index.ipynb)
- [Expected goals: Understand the math behind it](https://datascientest.com/en/expected-goals-understand-the-math-behind-it)
- [What is Data Analysis? An Expert Guide With Examples](https://www.datacamp.com/blog/what-is-data-analysis-expert-guide)